In [59]:
import logging
from collections import namedtuple
import random
from typing import Callable
from copy import deepcopy
from itertools import accumulate
from operator import xor

In [60]:
Nimply = namedtuple("Nimply", "row, num_objects")
random.seed(42)

In [61]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

In [62]:
def pure_random(state: Nim) -> Nimply:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

In [63]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))

In [64]:
def nim_sum(state: Nim) -> int:
    *_, result = accumulate(state.rows, xor)
    return result


def cook_status(state: Nim) -> dict:
    cooked = dict()
    # cooked["possible_moves"] = [
    #     (r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k
    # ]
    cooked["active_rows_number"] = sum(o > 0 for o in state.rows)
    cooked["shortest_row"] = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    cooked["longest_row"] = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    cooked["even_rows"] = [x[0] for x in enumerate(state.rows) if x[1] % 2 == 0 and x[1] > 0] 
    cooked["odd_rows"] = [x[0] for x in enumerate(state.rows) if x[1] % 2 != 0]
    cooked["active_rows"] = [x[0] for x in enumerate(state.rows) if x[1] > 0]
    cooked["sum"] = sum(state.rows)
    # cooked["nim_sum"] = nim_sum(state)

    # brute_force = list()
    # for m in cooked["possible_moves"]:
    #     tmp = deepcopy(state)
    #     tmp.nimming(m)
    #     brute_force.append((m, nim_sum(tmp)))
    # cooked["brute_force"] = brute_force

    return cooked

In [65]:
def optimal_startegy(state: Nim) -> Nimply:
    data = cook_status(state)
    return next((bf for bf in data["brute_force"] if bf[1] == 0), random.choice(data["brute_force"]))[0]

In [66]:
def dumb(state: Nim) -> Nimply:
    data = cook_status(state)
    i = random.choice(data["active_rows"])
    return Nimply(i,1)

In [67]:
def make_strategy(genome: dict) -> Callable:
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)

        # select the only rows that satisfy a certain thresshold 
        possible_choices = [i for i in data["active_rows"] if (data["sum"] - state.rows[i])/data["sum"] < genome["t"]]
        if len(possible_choices) == 0:
            i = random.choice(data["active_rows"])
        else:
            #i = random.choice(possible_choices)
            i = max(possible_choices, key=lambda x:state.rows[x])
        ply = Nimply(i, state.rows[i])       
        return ply

    return evolvable

In [68]:
# def make_strategy(genome: dict) -> Callable:
#     def evolvable(state: Nim) -> Nimply:
#         data = cook_status(state)

#         # even odd strategy 
#         if random.random() < genome["e"]:
#             if len(data["even_rows"]) > 0:
#                 first = data["even_rows"][0][0]
#                 ply = Nimply(first, state.rows[first])
#             else:
#                 first = data["odd_rows"][0][0]
#                 ply = Nimply(first, state.rows[first])
#         else :
#             if len(data["even_rows"]) > 0:
#                 first = data["even_rows"][0][0]
#                 ply = Nimply(first, state.rows[first])
#             else:
#                 first = data["odd_rows"][0][0]
#                 ply = Nimply(first, state.rows[first])

#         if random.random() < genome["p"]:
#             # shortest longest strategy
#             # ply = Nimply(data["shortest_row"], random.randint(1, state.rows[data["shortest_row"]]))
#             i = random.choice(data["active_rows"])
#             ply = Nimply(i, state.rows[i])       
#         else:
#             # shortest longest strategy
#             # ply = Nimply(data["longest_row"], random.randint(1, state.rows[data["longest_row"]])
            
#             # 1 element strategy
#             i = random.choice(data["active_rows"])
#             ply = Nimply(i, 1) 
            
#         return ply

#     return evolvable

In [69]:
NUM_MATCHES = 100
NIM_SIZE = 11

def evaluate(strategy: Callable, opponent_strategy: Callable = pure_random) -> float:
    opponent = (strategy, opponent_strategy)
    won = 0
    for m in range(NUM_MATCHES):
        nim = Nim(NIM_SIZE)
        player = 0
        while nim:
            ply = opponent[player](nim)
            nim.nimming(ply)
            player = 1 - player
        if player == 1:
            won += 1
    return won / NUM_MATCHES

In [70]:
Individual = namedtuple("Individual", ["genome", "fitness"])

def tournament(population, tournament_size=2):
    return max(random.sample(population,tournament_size), key=lambda i: i.fitness)



def tweak(g):
    if random.choice([0,1]) == 1:
        # increase value
        if g["t"] < 0.95: 
            g["t"] += 0.05
    else:
        # decrease value
        if g["t"] > 0.05: 
            g["t"] -= 0.05
    return g
    

In [71]:
NUM_GENERATIONS = 100
POPULATION_SIZE = 50
OFFSPRING_SIZE = 20
TOURNAMENT_SIZE = 2

def genetic_algorithm():
    # create the initial population
    population = list()
    for genome in [{"t" : random.random()} for _ in range(POPULATION_SIZE)]:
        population.append(Individual(genome, evaluate(make_strategy(genome))))
        
    # evolution
    for g in range(NUM_GENERATIONS):
        offspring = list()
        for _ in range(OFFSPRING_SIZE):
            p = tournament(population, tournament_size=TOURNAMENT_SIZE)
            o = tweak(p.genome)
            f = evaluate(make_strategy(o))
            offspring.append(Individual(o, f))
        population += offspring
        # selection 
        population = sorted(population, key=lambda i: i.fitness, reverse=True)[:POPULATION_SIZE]
        #print(f"Generation {g}: {population[0].genome} {population[0].fitness}")

    return population

In [72]:
population = genetic_algorithm()
print(f"Found solution {population[0].genome}, {population[0].fitness}")

Found solution {'t': 0.47979197692362746}, 0.85


In [75]:
final_parameter = population[0].genome
final_stategy = make_strategy(final_parameter)
print(f"{final_parameter} {evaluate(final_stategy)}")

{'t': 0.47979197692362746} 0.7


In [76]:
print(evaluate(final_stategy,gabriele))

1.0
